In [ ]:
import numpy as np
import keras2onnx
import onnxruntime
from keras.models import Sequential, load_model, Model
from keras.datasets import mnist
from metrics_face import *
import tensorflow.compat.v1 as tf
from glob import glob
import os
import facenet
from keras.preprocessing import image
from tqdm import tqdm
import keras

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()

In [ ]:
from PIL import Image
import cv2

In [ ]:
(X, y), (X_test, y_test) = mnist.load_data()
X = X[:, :, :, np.newaxis].astype('float32') / 255
X_test = X_test[:, :, :, np.newaxis].astype('float32') / 255

In [ ]:
def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(160, 160))
    return np.expand_dims(np.asarray(img),axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [ ]:
classes, nrof_classes = facenet.get_dataset("vggface2_train_2/train")
path_exp = os.path.expanduser("vggface2_train_2/train")
image_paths = [glob(os.path.join(path_exp, classes[i], "*.jpg")) for i in range(nrof_classes)]
split = int(round(nrof_classes*(1-0.99)))
X_test = paths_to_tensor([image_paths[0][0],image_paths[0][1]])

In [ ]:
classes, nrof_classes = facenet.get_dataset("vggface2_test_2/test")
nrof_classes = 30
np.random.shuffle(classes)
path_exp = os.path.expanduser("vggface2_test_2/test")
image_paths = [sorted(glob(os.path.join(path_exp, classes[i], "*.jpg")))[3:4] for i in range(nrof_classes)]
images = []
image_paths = np.array(image_paths).flatten().tolist()
for img_path in tqdm(image_paths):
    img = Image.open(img_path).convert('L')
    face_cascade = cv2.CascadeClassifier('/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml')
    img = np.asarray(img)
    faces_multi = face_cascade.detectMultiScale(img, 1.3, 5)
    faces = [face.astype(np.int64).tolist() for face in faces_multi]
    if len(faces) > 0:
        yes_face = faces[0]
        img = cv2.resize(img[yes_face[1]:yes_face[1]+yes_face[3],yes_face[0]:yes_face[0]+yes_face[2]],(160,160))
        images.append(img)

In [ ]:
arcface_model = load_model('models/mnist_vgg8_arcface_5d/model_sm.hdf5', custom_objects={'ArcFace': ArcFace})

In [ ]:
arcface_model1 = Model(inputs=arcface_model.input[0], outputs=arcface_model.layers[-4].output)
arcface_features1_array = []
for img in tqdm(images):
    arcface_features1 = arcface_model1.predict(img.reshape(1,160,160,1), verbose=1)
    arcface_features1_array.append(arcface_features1/np.linalg.norm(arcface_features1, axis=1, keepdims=True))

In [ ]:
class_weights = [1, 0.5, 1, 10, 10]
def intermediate_model(x):
    return keras.backend.max(x, axis=1) * \
(keras.activations.elu((x-keras.backend.mean(x, axis=1))/keras.backend.square(keras.backend.std(x)))+\
    keras.backend.square(x-keras.backend.mean(x, axis=1))/keras.backend.std(x)) + \
keras.backend.min(x, axis=1) * keras.backend.log(keras.backend.square(x-keras.backend.mean(x, axis=1)))
intermediary_array = []
for i in tqdm(range(len(images))):
    a = np.multiply(class_weights, intermediate_model(
        tf.convert_to_tensor(tf.convert_to_tensor(arcface_features1_array[i], dtype=tf.float32))).numpy())
    if i < 10:
        print(a)
    intermediary_array.append(a)

In [ ]:
import numpy as np

In [ ]:
np.__version__

In [1]:
import pickle

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 2: ordinal not in range(128)

In [ ]:
intermediary_array = np.dstack(intermediary_array)

In [ ]:
[ 0.11949151, -0.7965891 ,  0.26912665,  0.44944108,  0.27702197]
[ 0.1343845 , -0.6959256 ,  0.10052601, -0.57591254,  0.39477533]
[-0.20688158,  0.79289556, -0.17721854,  0.4884938 , -0.24183482]
[-0.23509903,  0.7068529 , -0.20450296,  0.50083554, -0.39042255]
[-0.27845502,  0.6834492 , -0.0885547 , -0.2526008 , -0.61944395]

In [ ]:
gamma = arcface_model.layers[-3].weights[0]
beta = arcface_model.layers[-3].weights[1]
moving_mean = arcface_model.layers[-3].weights[2]
moving_variance = arcface_model.layers[-3].weights[3]

In [ ]:
gamma.numpy(), beta.numpy(), moving_mean.numpy(), moving_variance.numpy()

In [ ]:
xNorm = (intermediary_array[0,:,:] - moving_mean.numpy().reshape(-1,1)) / np.sqrt(0.001 + moving_variance.numpy().reshape(-1,1))
data = gamma.numpy().reshape(-1,1) * xNorm + beta.numpy().reshape(-1,1)
np.round(data.T/np.linalg.norm(data.T,1),7)

In [ ]:
arcface_features2_array = np.dstack(arcface_features2_array)

In [ ]:
arcface_model2 = Model(inputs=arcface_model.input[0], outputs=arcface_model.layers[-3].output)
arcface_features2_array = []
for img in tqdm(images):
    arcface_features2 = arcface_model2.predict(img.reshape(1,160,160,1), verbose=1)
    arcface_features2_array.append(arcface_features2/np.linalg.norm(arcface_features2, axis=1, keepdims=True))

In [ ]:
p1 = [-0.176519  , -0.105028  ,  0.95071614, -0.21184155, -0.09524728]
p2 = [-0.17389667, -0.10519306,  0.93706113, -0.26787877, -0.09408321]
p3 = [-0.17534307, -0.09559851,  0.9498372 , -0.2157349 , -0.1066931 ]
p4 = [-0.17858912, -0.09505837,  0.9690808 , -0.09122588, -0.10784248]
p5 = [-0.17978127, -0.1065986 ,  0.9706176 , -0.04294081, -0.11123333]

In [ ]:
print(librosa.istft(np.dstack([p1]*5).reshape(-1,5), hop_length=1, win_length=1))
print(librosa.istft(np.dstack([p2]*5).reshape(-1,5), hop_length=1, win_length=1))
print(librosa.istft(np.dstack([p3]*5).reshape(-1,5), hop_length=1, win_length=1))
print(librosa.istft(np.dstack([p4]*5).reshape(-1,5), hop_length=1, win_length=1))
print(librosa.istft(np.dstack([p5]*5).reshape(-1,5), hop_length=1, win_length=1))

In [ ]:
from sklearn.datasets import load_iris
from pandas.plotting import andrews_curves

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

# Look pretty...
matplotlib.style.use('ggplot')
# If the above line throws an error, use plt.style.use('ggplot') instead

# Load up SKLearn's Iris Dataset into a Pandas Dataframe
class_weights = [0.1, 0.5, 0.1, 1, 1]
df = pd.DataFrame(
    np.multiply(class_weights, 
      np.exp(arcface_features1_array-np.expand_dims(arcface_features1_array.mean(axis=1),1)/\
      np.expand_dims(arcface_features1_array.std(axis=1),1)) + \
    np.expand_dims(arcface_features1_array.mean(axis=1),1)
   ), columns=['col1','col2','col3','col4','col5'])
df['target_names'] = [librosa.istft(
    np.dstack([df.iloc[i,:].values]*5).reshape(-1,5), hop_length=1, win_length=1
).mean() for i in range(len(df))]
# Andrews Curves Start Here:
plt.figure()
andrews_curves(df, 'target_names')
plt.show()

In [ ]:
[-0.17978127, -0.1065986 ,  0.9706176 , -0.04294081, -0.11123333]
[-0.17668884, -0.11009184,  0.93734115, -0.26282188, -0.09474736]
[-0.1755625 , -0.05969939,  0.97241575, -0.011096  , -0.14106144]
[-0.17391324, -0.05885269,  0.9708331 ,  0.01058049, -0.15382384]
[-0.17508644, -0.07741634,  0.96141636, -0.14232956, -0.13701247]

In [ ]:
p1 = [-0.176519  , -0.105028  ,  0.95071614, -0.21184155, -0.09524728]
p2 = [-0.17389667, -0.10519306,  0.93706113, -0.26787877, -0.09408321]
p3 = [-0.17534307, -0.09559851,  0.9498372 , -0.2157349 , -0.1066931 ]
p4 = [-0.17858912, -0.09505837,  0.9690808 , -0.09122588, -0.10784248]
p5 = [-0.17978127, -0.1065986 ,  0.9706176 , -0.04294081, -0.11123333]

In [ ]:
from sklearn.datasets import load_iris
from pandas.tools.plotting import andrews_curves

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

# Look pretty...
matplotlib.style.use('ggplot')
# If the above line throws an error, use plt.style.use('ggplot') instead

# Load up SKLearn's Iris Dataset into a Pandas Dataframe
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target_names'] = [data.target_names[i] for i in data.target]

In [ ]:
np.linalg.norm(arcface_features2_array, axis=1, keepdims=True)

In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
imagehash.average_hash(
    Image.open("vggface2_test/test/n000029/0130_01.jpg")
)

In [ ]:
import imagehash, cv2
from PIL import Image

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_generator = datagen.flow_from_directory(
        'data/train',
        target_size=(1580, 862),
        batch_size=32)
validation_generator = datagen.flow_from_directory(
        'data/validation',
        target_size=(1580, 862),
        batch_size=32,
        class_mode='binary')


In [ ]:
fig2 = plt.figure()
ax2 = Axes3D(fig2)
for c in range(len(np.unique(y_test))):
    ax2.plot(arcface_features[y_test==c, 0], arcface_features[y_test==c, 1], arcface_features[y_test==c, 2], '.', alpha=0.1)
plt.title('ArcFace')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
arcface_features

In [ ]:
arcface_features

In [ ]:
arcface_features

In [ ]:
arcface_features

In [ ]:
arcface_features

In [ ]:
print(librosa.istft(np.array([p1]*3).reshape(-1,3), hop_length=1, win_length=3))
print(librosa.istft(np.array([p2]*3).reshape(-1,3), hop_length=1, win_length=3))
print(librosa.istft(np.array([p3]*3).reshape(-1,3), hop_length=1, win_length=3))
print(librosa.istft(np.array([p4]*3).reshape(-1,3), hop_length=1, win_length=3))
print(librosa.istft(np.array([p5]*3).reshape(-1,3), hop_length=1, win_length=3))

In [ ]:
import librosa